In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Load 'bookings' dataset
bookings = pd.read_csv('Datasets\Bookings.csv')

In [ ]:
# Inspect first five rows of 'bookings' dataset including column title
bookings.head()

In [5]:
# Load 'sessions' dataset
sessions = pd.read_csv('Datasets\Sessions.csv')

In [ ]:
# Inspect first five rows of 'sessions' dateset including column title
sessions.head()

1. Find the number of distinct bookings, sessions, and searches from the given data
sets.


In [6]:
# To return the number of distinct bookings,  sessions and searches, we will use the 
# "nunique() method" of the pandas DataFrame
distinct_bookings = bookings['booking_id'].nunique()
distinct_sessions = sessions['session_id'].nunique()
distinct_searches = sessions['search_id'].nunique()

In [ ]:
# Print out the number of distinct bookings, sessions and searches. 
print(f"Number of Distinct Bookings: {distinct_bookings}")
print(f"Number of Distinct Sessions: {distinct_sessions}")
print(f"Number of Distinct Searches: {distinct_searches}")

2. How many sessions have more than one booking?

In [ ]:
# First, we will group the bookings by session_id and then count the number of unique bookings in each session
bookings_per_session = sessions.groupby('session_id')['booking_id'].nunique()

# Secondly, we will filter out sessions where the count of bookings < 1
sessions_with_multiple_bookings = bookings_per_session[bookings_per_session > 1]

# Thirdly, we will then get the count of these sessions
number_of_sessions_with_multiple_bookings = sessions_with_multiple_bookings.count()

# Now, let's print out the final result
print(f"The total number of sessions with more than one booking: {number_of_sessions_with_multiple_bookings}")


3. Which days of the week have the highest number of bookings? Also, draw a pie
chart to show the distribution for all days of the week?

In [ ]:
# Let's convert the 'booking_time' to datetime format
bookings['booking_time'] = pd.to_datetime(bookings['booking_time'])

# Extract the day of the week from the 'booking_time'
bookings['day_of_week'] = bookings['booking_time'].dt.day_name()

# Count the number of bookings for each day of the week
bookings_per_day = bookings['day_of_week'].value_counts()

# Find the day(s) with the highest number of bookings
highest_bookings_day = bookings_per_day.idxmax()
print("Booking counts for all days of the week:")
print(f"{bookings_per_day} \n")
print(f"The day with the highest number of bookings: {highest_bookings_day} \n")


# Plot a pie chart for the distribution of bookings across all days
plt.figure(figsize=(6, 6))
bookings_per_day.plot.pie(autopct='%1.1f%%', startangle=90, explode = (0.3, 0, 0, 0, 0,0, 0), colors=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#ff9864', '#73B22D', '#FF7800', '#5f5d60'])
plt.title("Distribution of Bookings by Day of the Week")
plt.ylabel("")  # Remove y-axis label for better aesthetics
plt.show()

4. For each of the service names, display the total number of bookings and the total
Gross Booking Value in INR.

In [ ]:
# Group by 'service_name' and calculate total bookings and total INR_Amount
service_summary = bookings.groupby('service_name').agg(
    Num_of_bookings=('booking_id', 'count'),
    total_gross_value=('INR_Amount', 'sum')
).reset_index()

# Display the summary
print("Service Name Summary:")
print(service_summary)


5.  For customers who have more than 1 booking, which is the most booked route
(from_city to to_city)?

In [ ]:
# Group by 'customer_id' and count the number of bookings for each customer
customer_booking_counts = bookings.groupby('customer_id')['booking_id'].count()


# Filter for customers with more than 1 booking
customers_with_multiple_bookings = customer_booking_counts[customer_booking_counts > 1].index


# Filter the bookings data to include only these customers
filtered_bookings = bookings[bookings['customer_id'].isin(customers_with_multiple_bookings)].copy()

# Create a route column combining 'from_city' and 'to_city' using .loc
filtered_bookings.loc[:, 'route'] = filtered_bookings['from_city'] + " to " + filtered_bookings['to_city']

# Find the most booked route
most_booked_route = filtered_bookings['route'].value_counts().idxmax()

# Print the result
print(f"The most booked route among customers with more than 1 booking is: {most_booked_route}")


6. Which are the top 3 departure cities from where customers book mostly in advance,
provided that there have been at least 5 departures from that city?

In [ ]:
# Group by departure city
grouped_by_dep_city=bookings.groupby(['to_city']).agg({'days_to_departure':['mean', 'count']}).reset_index()

# Label column headings
grouped_by_dep_city.columns=['DepCity', 'Avg_days_to_dep', 'Total_departures']

# Display the result
print("Top 3 departure cities where customers book mostly in advance (with at least 5 departures):")
print(grouped_by_dep_city[grouped_by_dep_city['Total_departures']>=5].sort_values('Avg_days_to_dep', ascending=False).head(3))


7. Plot a heatmap displaying correlations of the numerical column and report which
pair of numerical columns in the bookings data set, have the maximum correlation?

In [ ]:
# let's select the numerical columns from the bookings dataset for our correlation analysis
numerical_columns = bookings.select_dtypes(include=['float64']).columns

# Let us compute the correlation matrix for the selected numerical columns
correlation_matrix = bookings[numerical_columns].corr()

# Now, let's plot the heatmap of our correlation matrix
plt.figure(figsize=(6, 6))
sns.heatmap(correlation_matrix, annot=True, vmax=1, vmin=-1, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title("Heatmap showing Correlation of Numerical Columns in the Bookings Dataset")
plt.show()

# Let's find the pair of numerical columns with the maximum correlation
max_corr_value = correlation_matrix.where(~correlation_matrix.isna()).unstack().max()
max_corr_pair = correlation_matrix.stack().idxmax()

# Display the result
print(f"The pair of numerical columns with the maximum correlation is: {max_corr_pair}")
print(f"Maximum correlation value: {max_corr_value}")


8. For each service, which is the most used device type for making bookings on the
platform?

In [ ]:
# Group by 'service_name' and 'device_type_used' and count the number of bookings for each combination
device_usage_per_service = bookings.groupby(['service_name', 'device_type_used'])['booking_id'].count()

# For each service, find the device type with the maximum count of bookings
most_used_device_per_service = device_usage_per_service.groupby('service_name').idxmax()

# Display the result
print("Most used device type for each service:")
print(most_used_device_per_service)


9. Plot the trends at a quarterly frequency for the number of bookings by each of the
device types, that is, plot a time series for each year and quarter showing the
number of bookings performed by each device type.

In [ ]:
# Ensure 'booking_time' is timezone-naive before converting to a period

bookings['booking_time'] = pd.to_datetime(bookings['booking_time']).dt.tz_localize(None)

# Extract year and quarter from 'booking_time'
bookings['year_quarter'] = bookings['booking_time'].dt.to_period('Q')

# Group by 'year_quarter' and 'device_type_used', and count the number of bookings
bookings_by_device = bookings.groupby(['year_quarter', 'device_type_used'])['booking_id'].count().unstack()

# View the first few rows of results
bookings[['year_quarter', 'booking_id', 'device_type_used']].head()


In [ ]:
# Now let's plot the Time series for each year and quarter showing the number of bookings performed by each device type.

bookings_by_device = bookings.groupby(['year_quarter', 'device_type_used'])['booking_id'].count().unstack()
# Check if there is data to plot
if not bookings_by_device.empty:
    plt.figure(figsize=(10, 6))
    bookings_by_device.plot(kind='line', marker='o')
    plt.title('Quarterly Trends of Bookings by Device Type')
    plt.xlabel('Year and Quarter')
    plt.ylabel('Number of Bookings')
    plt.grid(True)
    plt.legend(title='Device Type')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No data available to plot.")


10. Consider the following example:

    12 customers performed a total of 100 searches but only a few of them performed 10 bookings in total. <br><br>
        Searches = 100 &nbsp;&nbsp;&nbsp;
        bookings = 10 &nbsp;&nbsp;&nbsp;
        customers = 12
        
    The overall booking to search ratio, Ober, is 10/100 = 0.1 <br>
    Using the above information/context, answer the following:<br>
&nbsp;&nbsp;&nbsp;  • What is the average oBSR for each month of the year?<br>
&nbsp;&nbsp;&nbsp;  • What is the average oBSR for each day of the week?<br>
&nbsp;&nbsp;&nbsp;  • Plot a time series of oBSR on all the given dates.


In [16]:
# Convert 'booking_time' and 'search_time' to datetime format
bookings['booking_time']=pd.to_datetime(bookings['booking_time'], format='ISO8601')
sessions['search_time']=pd.to_datetime(sessions['search_time'], format='ISO8601')

# Count searches per day for each day (assuming there's a 'search_time')
search_counts = sessions.groupby(sessions['search_time'].dt.date)['search_id'].count()

# Count bookings per day for each day (assuming there's a 'booking_time')
booking_counts = bookings.groupby(bookings['booking_time'].dt.date)['booking_id'].count()

# Merge the booking counts and search counts by date
merged_counts = pd.DataFrame({
    'searches': search_counts,
    'bookings': booking_counts
}).fillna(0)  # Fill NaNs with 0 if no searches/bookings occurred on a given day

# Calculate oBSR for each day (Booking-to-Search Ratio)
merged_counts['oBSR'] = merged_counts['bookings'] / merged_counts['searches']

10.a. What is the average oBSR for each month of the year?

In [ ]:
merged_counts['month'] = pd.to_datetime(merged_counts.index).month_name()
average_oBSR_month = merged_counts.groupby('month')['oBSR'].mean()

print("Average oBSR for each month of the year:")
print(average_oBSR_month)

10.b.  What is the average oBSR for each day of the week?

In [ ]:
merged_counts['day_of_week'] = pd.to_datetime(merged_counts.index).day_name()
average_oBSR_weekday = merged_counts.groupby('day_of_week')['oBSR'].mean()

print("\nAverage oBSR for each day of the week:")
print(average_oBSR_weekday)

10.c. Plot a time series of oBSR on all the given dates.

In [ ]:

plt.figure(figsize=(10, 6))
merged_counts['oBSR'].plot(kind='line', marker='o')
plt.title('Time Series of Booking to Search Ratio (oBSR)')
plt.xlabel('Date')
plt.ylabel('oBSR (Booking to Search Ratio)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()